In [15]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("postgresql+psycopg2://localhost:5430/retain_dev")

streaming_events = pd.read_sql(
    "SELECT * FROM streaming_events",
    engine
)

streaming_events.head()


,event_id,account_id,event_timestamp,content_id,device_type,watch_duration_minutes,login_lat,login_long
0,EVT_BE66F608899D,ACC_728CCBB1,2025-07-15 14:16:19,CNT_2BAF3B,mobile,35.6,33.4844,-112.3073
1,EVT_8FF66F8611DF,ACC_728CCBB1,2025-08-26 21:42:52,CNT_A0D0C1,mobile,9.8,33.5955,-112.1033
2,EVT_60A4517E5DF9,ACC_728CCBB1,2024-07-28 16:32:19,CNT_C467B2,smart_tv,36.2,33.3834,-111.9601
3,EVT_7A803BD5EE78,ACC_728CCBB1,2024-08-31 11:49:11,CNT_EEC577,smart_tv,30.4,33.2141,-112.1631
4,EVT_47AECD5C2345,ACC_728CCBB1,2024-09-09 15:37:19,CNT_1A0157,smart_tv,35.5,33.3966,-112.1560


In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


: 

In [ ]:
# Ensure timestamp is datetime
streaming_events['event_timestamp'] = pd.to_datetime(streaming_events['event_timestamp'])

# Time-based features
streaming_events['date'] = streaming_events['event_timestamp'].dt.date
streaming_events['hour'] = streaming_events['event_timestamp'].dt.hour
streaming_events['day_of_week'] = streaming_events['event_timestamp'].dt.day_name()

streaming_events.head()


In [ ]:
#Usage Over Time (Daily Event Volume)
daily_events = (
    streaming_events
    .groupby('date')
    .size()
    .reset_index(name='event_count')
)

fig = px.line(
    daily_events,
    x='date',
    y='event_count',
    title='Daily Streaming Events',
    markers=True
)

fig.show()


In [ ]:
#Device Type Usage Share
device_counts = (
    streaming_events
    .groupby('device_type')
    .size()
    .reset_index(name='events')
)

fig = px.pie(
    device_counts,
    names='device_type',
    values='events',
    title='Streaming Events by Device Type',
    hole=0.4
)

fig.show()


In [ ]:
#Average Watch Duration by Device
avg_watch_by_device = (
    streaming_events
    .groupby('device_type')['watch_duration_minutes']
    .mean()
    .reset_index()
)

fig = px.bar(
    avg_watch_by_device,
    x='device_type',
    y='watch_duration_minutes',
    title='Average Watch Duration by Device',
    text_auto='.2f'
)

fig.show()


In [ ]:
#Hourly Viewing Patterns (Time-of-Day Heat)
hourly_events = (
    streaming_events
    .groupby('hour')
    .size()
    .reset_index(name='events')
)

fig = px.line(
    hourly_events,
    x='hour',
    y='events',
    title='Streaming Activity by Hour of Day',
    markers=True
)

fig.show()


In [ ]:
# Day of Week Engagement
dow_events = (
    streaming_events
    .groupby('day_of_week')
    .size()
    .reindex([
        'Monday','Tuesday','Wednesday',
        'Thursday','Friday','Saturday','Sunday'
    ])
    .reset_index(name='events')
)

fig = px.bar(
    dow_events,
    x='day_of_week',
    y='events',
    title='Streaming Events by Day of Week'
)

fig.show()


In [ ]:
#Top Content by Total Watch Time
top_content = (
    streaming_events
    .groupby('content_id')['watch_duration_minutes']
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

fig = px.bar(
    top_content,
    x='content_id',
    y='watch_duration_minutes',
    title='Top 10 Content by Total Watch Time'
)

fig.show()


In [ ]:
#User Engagement Distribution
user_events = (
    streaming_events
    .groupby('account_id')
    .size()
    .reset_index(name='event_count')
)

fig = px.histogram(
    user_events,
    x='event_count',
    nbins=30,
    title='Distribution of Events per Account',
    marginal='box'
)

fig.show()


In [ ]:
#Geo Heatmap (Where Users Watch)
fig = px.density_mapbox(
    streaming_events,
    lat='login_lat',
    lon='login_long',
    radius=125,
    zoom=8,
    mapbox_style='carto-positron',
    title='Geographic Density of Streaming Events'
)

fig.show()


In [ ]:
#Session Length vs Time of Day
fig = px.box(
    streaming_events,
    x='hour',
    y='watch_duration_minutes',
    title='Watch Duration by Hour of Day'
)

fig.show()
